**SCRAPPING**

Dans cette première étape, nous allons effectuer du scrapping sur le site Monster.fr

Celui-ci référence les emplois, le salaires, les zones sur une seule et même page.

Il n'est pas nécéssaire de changer de page pour afficher plus d'offres, les emplois s'actualisent automatiquement en scrollant en bas de page.

**Ce code a donc pour but de :**
- Se rendre sur le site Monster.fr, section Data scientist
- Scroller automatiquement en bas de page pendant X secondes (modifiable dans le code)
- Collecter les données et les ranger dans un tableau via pandas (Nom des posts, zones, salaires, mois en cours, années en cours)
- Créer un fichier Excel avec les données collectées

Vous pouvez run all directement

In [7]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
%pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
from datetime import datetime

def extract_salary(salary_str):
    numbers = re.findall(r'\d+', salary_str)
    return ''.join(numbers) if numbers else None

# Initialisation du pilote Selenium
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Accès à la page
url = "https://www.monster.fr/emploi/recherche?q=data+scientist&where=france&page=3&so=m.h.s"
driver.get(url)

# Exécution d'un script JavaScript pour faire défiler la fenêtre du navigateur
for _ in range(50):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

    try:
        li_elements = driver.find_elements(By.CSS_SELECTOR, 'ul.sc-harTkY.jEHPnr > li.sc-blKGMR.etPslv')
        for li in li_elements:
            driver.execute_script("arguments[0].scrollIntoView();", li)
            time.sleep(1)
    except Exception as e:
        print("Erreur lors du défilement des éléments <li> :", e)

time.sleep(1)

# Collecte des données
data = []
today = datetime.today()
current_month = today.strftime('%m')
current_year = today.strftime('%Y')
job_listings = driver.find_elements(By.CSS_SELECTOR, 'article[data-testid="svx_jobCard"]')
for job in job_listings:
    job_title_elements = job.find_elements(By.CSS_SELECTOR, 'a[data-testid="jobTitle"]')
    company_elements = job.find_elements(By.CSS_SELECTOR, 'span.sc-kufkCr.gQYLTh')
    location_elements = job.find_elements(By.CSS_SELECTOR, 'span[data-testid="jobDetailLocation"]')
    salary_elements = job.find_elements(By.CSS_SELECTOR, 'span.sc-fKMpNL.dasswI.ds-tag-label')
    
    job_title = job_title_elements[0].text if job_title_elements else 'N/A'
    company = company_elements[0].text if company_elements else 'N/A'
    location = location_elements[0].text if location_elements else 'N/A'
    salary = extract_salary(salary_elements[0].text) if salary_elements else None
    # job_type = None
    # for elem in salary_elements:
    #     text = elem.text.strip()
    #     if text in ["CDI", "Temps plein", "Temps partiel", "Freelance", "Stage / Apprentissage", "Intérim / CDD"]:
    #         job_type = text
    #         break
    
    data.append([job_title, company, location, salary, current_month, current_year])

driver.quit()

# Création d'un DataFrame avec pandas
df = pd.DataFrame(data, columns=["Job Title", "Company", "Location", "Salary", "Month", "Year"])

# Traitement des valeurs de salaire
def process_salary(salary_str):
    if salary_str and len(salary_str) == 10:
        salary_min = int(salary_str[:5])
        salary_max = int(salary_str[5:])
    else:
        salary_min = None
        salary_max = None
    return salary_min, salary_max

df[['Salary min', 'Salary max']] = df.apply(lambda row: process_salary(row['Salary']), axis=1, result_type='expand')
df['Salary'] = df[['Salary min', 'Salary max']].mean(axis=1)

# Réordonner les colonnes pour inclure 'Job Type'
column_order = ["Job Title", "Company", "Location", "Salary", "Salary min", "Salary max", "Month", "Year"]
df = df[column_order]

# Suppression des lignes où le salaire est 0 ou non disponible
df = df.dropna(subset=['Salary'])

# Enregistrement dans un fichier Excel
df.to_excel("jobListrange50.xlsx", index=False)


KeyboardInterrupt: 